In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('../input/titanic/train.csv')

In [ ]:
df1.head()

In [ ]:
df1.drop(['PassengerId','Name','Ticket','Cabin','Embarked','SibSp','Parch'], axis=1, inplace=True )

In [ ]:
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
df1['Age'].describe()

In [ ]:
df1['Age'].fillna(df1['Age'].mean(),inplace=True)

In [ ]:
df1.isnull().sum()

In [ ]:
l_sex_dummies=pd.get_dummies(df1['Sex'],drop_first=True)

In [ ]:
df1= pd.concat([df1,l_sex_dummies],axis=1)

In [ ]:
df1.head()

In [ ]:
df1.drop(['Sex'], axis=1, inplace=True )

In [ ]:
df1.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sts =StandardScaler()

In [ ]:
feature_scale = ['Age','Fare']
df1[feature_scale] = sts.fit_transform(df1[feature_scale])

In [ ]:
df1.head()

In [ ]:
X=df1.drop(['Survived'],axis=1)
y=df1['Survived']

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
#create param
model_param = {
    'DecisionTreeClassifier':{
        'model':DecisionTreeClassifier(),
        'param':{
            'criterion': ['gini','entropy']
        }
    },
        'KNeighborsClassifier':{
        'model':KNeighborsClassifier(),
        'param':{
            'n_neighbors': [5,10,15,20,25]
        }
    },
        'SVC':{
        'model':SVC(),
        'param':{
            'kernel':['rbf','linear','sigmoid'],
            'C': [0.1, 1, 10, 100]
         
        }
    }
}

In [ ]:
scores =[]
for model_name, mp in model_param.items():
    model_selection = GridSearchCV(estimator=mp['model'],param_grid=mp['param'],cv=5,return_train_score=False)
    model_selection.fit(X,y)
    scores.append({
        'model': model_name,
        'best_score': model_selection.best_score_,
        'best_params': model_selection.best_params_
    })

In [ ]:
df_model_score = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_model_score

In [ ]:
model_svc = SVC( C= 100,kernel='rbf')

In [ ]:
model_svc.fit(X, y)

In [ ]:
df2 = pd.read_csv('../input/titanic/test.csv')

In [ ]:
df2.head()

In [ ]:
df3=df2.drop(['PassengerId','Name','Ticket','Cabin','Embarked','SibSp','Parch'], axis=1 )

In [ ]:
df3.isnull().sum()

In [ ]:
df3['Age'].fillna(df3['Age'].mean(),inplace=True)
df3['Fare'].fillna(df3['Fare'].mean(),inplace=True)

In [ ]:
l_sex_dummies=pd.get_dummies(df3['Sex'],drop_first=True)
df3= pd.concat([df3,l_sex_dummies],axis=1)
df3.drop(['Sex'], axis=1, inplace=True )

In [ ]:
df3.head()

In [ ]:
df3[feature_scale] = sts.fit_transform(df3[feature_scale])

In [ ]:
df3.head()

In [ ]:
y_predicted = model_svc.predict(df3)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": df2['PassengerId'],
        "Survived": y_predicted
    })

In [ ]:
submission.to_csv('titanic_submission_v02.csv', index=False)